ultimate data (down)loader from yfinance

In [22]:
import sqlite3
import pandas as pd
import datetime
import numpy as np
from matplotlib import pyplot as plt
import time

database = "../main.db"

## setup and connection to DB

In [192]:
conn.close() # comment this if running the first time
conn = sqlite3.connect(database)

In [8]:
above5 = pd.read_sql_query('SELECT distinct ticker FROM history where close>5.0', conn)
above5.to_csv('tickers_above_5.csv', index=False)
above5 = sorted(above5.ticker.to_list())

In [7]:
import yfinance as yf

## earnings: run every Q

dodać ticker!

In [36]:
existing_tickers = pd.read_sql_query('SELECT distinct ticker FROM yf_earnings', conn)['ticker'].to_list()
missing_tickers = list(set(above5).difference(set(existing_tickers)))

In [37]:
missing_tickers

['NWS', 'CPTK', 'FOX', 'STKH']

In [38]:
# run this loop first with above5 list of ticker. If any ticker is missing, rerun it with missing_tickers

for i,ticker in enumerate(missing_tickers): #(above5):
    print(i,len(above5),ticker)
    try:
        model = yf.Ticker(ticker)
        ed = model.earnings_dates
        if len(ed) == 0 or ed is None:
            print("waiting 1 min for API to respond...")
            time.sleep(60)
            ed = model.earnings_dates
        if len(ed) == 0 or ed is None:
            print("second trial failed. skipping...")
            continue
        
        ed['ticker'] = ticker
        ed.reset_index().to_sql('yf_earnings', conn, if_exists='append', index=False)
    except Exception as e:
        "ERROR: "+str(e)

0 1114 NWS
- NWS: No earnings dates found, symbol may be delisted
1 1114 CPTK
- CPTK: No earnings dates found, symbol may be delisted
2 1114 FOX
- FOX: No earnings dates found, symbol may be delisted
3 1114 STKH
- STKH: No earnings dates found, symbol may be delisted


In [46]:
start_date = pd.read_sql_query('SELECT DATE(min("Earnings Date")) FROM yf_earnings', conn).to_numpy()[0][0]
existing_tickers = pd.read_sql_query('SELECT distinct ticker FROM yf_earnings', conn)['ticker'].to_list()

getting history...

In [49]:
def create_table(conn, create_table_sql):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Exception as e:
        print(e)

In [50]:
sql_create_history_table = """ CREATE TABLE yf_history (
                                        id integer PRIMARY KEY,
                                        ticker text NOT NULL,
                                        trade_date date,
                                        open decimal,
                                        high decimal,
                                        low decimal,
                                        close decimal,
                                        volume integer
                                    ); """
create_table(conn, sql_create_history_table)

In [48]:
end_date   = datetime.datetime.now().strftime('%Y-%m-%d')
print(f"Getting tickers from {start_date} till {end_date}")

Getting tickers from 2019-01-31 till 2023-01-22


In [51]:
for i,ticker in enumerate(existing_tickers):
    print(f"{i}/{len(existing_tickers)} {ticker}...")
    try:
        tmp=yf.download(ticker, start_date, end_date)
    except Exception as e:
        print("Exception: "+str(e))
        continue
    
    tmp = tmp\
        .reset_index()\
        .drop(columns='Adj Close')\
        .rename(columns={
            "Date":"trade_date",
            "Open":"open",
            "Close":"close",
            "High":"high",
            "Low":"low",
            "Volume":"volume"
        })
    tmp['ticker'] = ticker
    
    tmp.to_sql('yf_history', conn, if_exists='append', index=False)

0/1110 A...
[*********************100%***********************]  1 of 1 completed
1/1110 AA...
[*********************100%***********************]  1 of 1 completed
2/1110 AAL...
[*********************100%***********************]  1 of 1 completed
3/1110 AAON...
[*********************100%***********************]  1 of 1 completed
4/1110 AAPL...
[*********************100%***********************]  1 of 1 completed
5/1110 ABBV...
[*********************100%***********************]  1 of 1 completed
6/1110 ABC...
[*********************100%***********************]  1 of 1 completed
7/1110 ABCL...
[*********************100%***********************]  1 of 1 completed
8/1110 ABM...
[*********************100%***********************]  1 of 1 completed
9/1110 ABMD...
[*********************100%***********************]  1 of 1 completed
10/1110 ABNB...
[*********************100%***********************]  1 of 1 completed
11/1110 ABT...
[*********************100%***********************]  1 of 1 completed

## other financials

In [125]:
missing_tickers = list(set(existing_tickers).difference(set(pd.read_sql_query('SELECT distinct ticker FROM yf_fin', conn)['ticker'].to_list())))

In [126]:
len(set(pd.read_sql_query('SELECT distinct ticker FROM yf_fin', conn)['ticker'].to_list()))

1106

In [127]:
missing_tickers

['HTPA', 'CLR', 'BAM', 'DRE']

In [128]:
for i,ticker in enumerate(missing_tickers):
    print(f"{i}/{len(existing_tickers)} {ticker}...")
    model = yf.Ticker(ticker)
    # balance sheet
    try:
        bs = model\
            .balance_sheet\
            .transpose()
        bs_cols = [c for c in bs.columns if c in ['Total Assets','Common Stock','Total Capitalization','Total Debt', 'Net Debt','Ordinary Shares Number']]
        bs = bs[bs_cols]
        # income statement
        ist = model\
            .income_stmt\
            .transpose()
        ist_cols = [c for c in ist.columns if c in ['Total Revenue','Gross Profit','Operating Expense','Operating Income','Basic Average Shares']]
        ist = ist[ist_cols]
    except Exception as e:
        print(e)
        continue
    # join the above
    fin = bs.join(ist).reset_index().rename(columns={'index':'yearly_date'})
    print(fin.shape)
    fin['ticker'] = ticker

    fin.to_sql('yf_fin', conn, if_exists='append', index=False)

    # break

0/1110 HTPA...
- HTPA: Failed to create balance-sheet financials table for reason: YFinanceDataException("Parsing FinancialTemplateStore failed, reason: KeyError('FinancialTemplateStore')")
- HTPA: Failed to create income financials table for reason: YFinanceDataException("Parsing FinancialTemplateStore failed, reason: KeyError('FinancialTemplateStore')")
(0, 1)
1/1110 CLR...
- CLR: Failed to create balance-sheet financials table for reason: YFinanceDataException("Parsing FinancialTemplateStore failed, reason: KeyError('FinancialTemplateStore')")
- CLR: Failed to create income financials table for reason: YFinanceDataException("Parsing FinancialTemplateStore failed, reason: KeyError('FinancialTemplateStore')")
(0, 1)
2/1110 BAM...
(0, 1)
3/1110 DRE...
- DRE: Failed to create balance-sheet financials table for reason: YFinanceDataException("Parsing FinancialTemplateStore failed, reason: KeyError('FinancialTemplateStore')")
- DRE: Failed to create income financials table for reason: YFin

## secondary tables

In [153]:
pd.read_sql_query('SELECT * FROM yf_earnings limit 5', conn).columns

Index(['Earnings Date', 'EPS Estimate', 'Reported EPS', 'Surprise(%)',
       'ticker'],
      dtype='object')

In [130]:
pd.read_sql_query('SELECT * FROM yf_history limit 5', conn)

,id,ticker,trade_date,open,high,low,close,volume
0,1,A,2019-01-31 00:00:00-05:00,75.209999,76.309998,74.669998,76.050003,2951300
1,2,A,2019-02-01 00:00:00-05:00,76.160004,76.589996,75.620003,76.040001,1343900
2,3,A,2019-02-04 00:00:00-05:00,76.180000,76.230003,75.320000,76.029999,2245900
3,4,A,2019-02-05 00:00:00-05:00,75.800003,76.510002,75.790001,76.309998,1593500
4,5,A,2019-02-06 00:00:00-05:00,76.110001,76.690002,76.000000,76.300003,1504900


In [152]:
pd.read_sql_query('SELECT * FROM yf_fin limit 5', conn)

,yearly_date,Total Assets,Common Stock,Total Capitalization,Total Debt,Net Debt,Ordinary Shares Number,Total Revenue,Gross Profit,Operating Expense,Operating Income,Basic Average Shares,ticker
0,2021-12-31 00:00:00,8.876400e+09,6.000000e+05,5.276800e+09,2.490400e+09,1.581900e+09,40170101.0,2.412400e+09,1.365300e+09,3.994000e+08,9.659000e+08,41500000.0,AMG
1,2020-12-31 00:00:00,7.888900e+09,6.000000e+05,5.091800e+09,2.312100e+09,1.272400e+09,44000000.0,2.027500e+09,1.143800e+09,5.338000e+08,6.100000e+08,46500000.0,AMG
2,2019-12-31 00:00:00,7.653500e+09,6.000000e+05,4.731300e+09,1.793800e+09,1.254200e+09,48100000.0,2.239600e+09,1.296600e+09,5.996000e+08,6.970000e+08,50500000.0,AMG
3,2021-12-31 00:00:00,6.116500e+10,3.209600e+10,3.992200e+10,3.330900e+10,2.532000e+10,558300000.0,2.597900e+10,1.952500e+10,1.038100e+10,9.144000e+09,570000000.0,AMGN
4,2020-12-31 00:00:00,6.294800e+10,3.180200e+10,4.230400e+10,3.298600e+10,2.672000e+10,578300000.0,2.542400e+10,1.926500e+10,1.012600e+10,9.139000e+09,586000000.0,AMGN


yf_constants

In [175]:
q = """
  CREATE TABLE yf_constants AS
  SELECT 
    *,
    LAG(eps_actual, 1) OVER (PARTITION BY ticker ORDER BY earnings_date ASC) prev_eps_actual,
    LAG(eps_diff, 1) OVER (PARTITION BY ticker ORDER BY earnings_date ASC) prev_eps_diff
  FROM (
    SELECT
        e.ticker,
        DATE(e."Earnings Date") earnings_date,
        DATE(f.yearly_date) financial_date,
        RANK() OVER (PARTITION BY e.ticker,"Earnings Date" ORDER BY yearly_date desc) rnk,
        e."EPS Estimate" eps_est, 
        e."Reported EPS" eps_actual, 
        e."Reported EPS"-e."EPS Estimate" eps_diff,
        e."Surprise(%)" eps_diff_pct,
        f."Total Assets" tot_assets, 
        f."Common Stock" common_stock, 
        f."Total Capitalization" tot_cap,
        f."Total Debt" tot_debt, 
        f."Net Debt" net_debt, 
        f."Ordinary Shares Number" ord_shares, 
        f."Total Revenue" tot_reve,
        f."Gross Profit" gross_profit, 
        f."Operating Expense" op_expense, 
        f."Operating Income" op_income,
        f."Basic Average Shares" avg_shares
    FROM yf_earnings e
    JOIN yf_fin f
      ON e.ticker=f.ticker
      AND earnings_date>=financial_date
    ) sub
    WHERE rnk=1
"""
conn.execute(q)
conn.commit()

In [193]:
q="""
CREATE TABLE yf_secondary_expanded AS
SELECT 
  c.*, 
  eps_actual-prev_eps_actual eps_q_change,
  (eps_actual-prev_eps_actual)/prev_eps_actual eps_q_change_pct,
  op_expense/op_income expense_to_income,
  tot_assets/gross_profit assets_to_profit,
  net_debt/tot_debt debt_to_debt,
  tot_debt/tot_cap debt_to_cap,
  gross_profit/tot_reve profit_to_reve,
  h.close as on_earnings_close,
  h.volume as on_earnings_volume,
  c.ord_shares * h.close as market_cap,
  date(earnings_date, '+1 day') as history_start_date,
  date(earnings_date, '+2 months') as history_end_date
FROM yf_constants c
JOIN yf_history h
  ON c.ticker=h.ticker
  AND c.earnings_date=DATE(h.trade_date)
WHERE eps_actual IS NOT NULL
  AND prev_eps_actual IS NOT NULL
"""
# df=pd.read_sql_query(q,conn)
# df.head(1).transpose()
conn.execute(q)
conn.commit()